<a href="https://colab.research.google.com/github/Bora-Ulu/Inflation-Technique/blob/main/Inflation_Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [318]:
from numpy import *
from itertools import *
import time
from scipy.sparse import coo_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix
from cvxopt import matrix, solvers, sparse, spmatrix
from numba import njit

@njit
def PrepForIntToArray32(ar):# Takes a huge column vector as input
                     # Returns a matrix where the rows are the base-4 binary expansions of each element of the input array
    #return transpose(atleast_2d(ravel(ar.astype(uint32)))).view(uint8)
    return reshape(ar.astype(uint32).view(uint8),(-1,4))


def IntToArray32(ar):
    return unpackbits(PrepForIntToArray32(ar),axis=1, bitorder='little').view(bool_)

@njit
def PrepForArrayToInt32(bitarray):
    as2mat=bitarray.reshape(-1, bitarray.shape[-1])
    temparray=zeros((len(as2mat),32),bool_)
    temparray[:,:bitarray.shape[-1]]=as2mat
    # return pad(bitarray.reshape(-1, bitarray.shape[-1]),[(0,0), (0, 32-bitarray.shape[-1])], mode='constant')
    return temparray

def ArrayToInt32(bitarray):
    
    return reshape(packbits(PrepForArrayToInt32(bitarray), bitorder='little').view(uint32),bitarray.shape[:-1])

ar=transpose(atleast_2d(arange(0,4**12,1,uint32))) 
arasbits=unpackbits(ar.view(uint8),axis=1, bitorder='little').view(bool_)

In [317]:
PrepForArrayToInt32(IntToArray32(ar)).shape

(67108864, 32)

In [322]:
ArrayToInt32(IntToArray32(ar))

array([       0,        1,        2, ..., 16777213, 16777214, 16777215],
      dtype=uint32)

In [281]:
SwapX1X2=arange(32);
SwapX1X2[23-array([0,1,2,3,4,5,6,7,8,9,12,13,10,11,14,15])]=SwapX1X2[23-array([2,3,0,1,6,7,4,5,12,13,8,9,14,15,10,11])];
SwapY1Y2=arange(32);
SwapY1Y2[23-array([0,1,4,5,2,3,6,7,16,17,18,19,20,21,22,23])]=SwapY1Y2[23-array([4,5,0,1,6,7,2,3,18,19,16,17,22,23,20,21])];
SwapZ1Z2=arange(32);
SwapZ1Z2[23-array([8,9,10,11,12,13,14,15,16,17,20,21,18,19,22,23])]=SwapZ1Z2[23-array([10,11,8,9,14,15,12,13,20,21,16,17,22,23,18,19])]

In [ ]:
"""
def Symmetry_Op(operation, ar):#Takes the exchange opration of copy latent variables and the values of the observable variables as a uint32 column vector as input
                                   #Returns the corresponding observable values for the resultant identical probability
    
    # The input list is of the form: [A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,C3,C4]
  
    A=unpackbits(ar.view(uint8), axis=1, bitorder='little').astype(bool_)
    
    if operation == 'X1_X2': #Exchanging \Lambda_{1}^{AB} and \Lambda_{2}^{AB} 
        return packbits(A[:,SwapX1X2], bitorder='little').view(uint32)
        
    elif operation == 'Y1_Y2': #Exchanging \Lambda_{1}^{AC} and \Lambda_{2}^{AC}
        return packbits(A[:,SwapY1Y2], bitorder='little').view(uint32)
        
    elif operation == 'Z1_Z2': #Exchanging \Lambda_{1}^{BC} and \Lambda_{2}^{BC}
        return packbits(A[:,SwapZ1Z2], bitorder='little').view(uint32)
    
    else:
        print('Error: Enter a valid operation.')
        
ar=transpose(atleast_2d(arange(0,4**12,1,uint32)))        
start = time.time()

A=Symmetry_Op('X1_X2',ar)
print('It took', time.time()-start, 'seconds.')
"""

In [ ]:
"""
B=vstack((A,A))
B.shape
"""

In [115]:
def Symmetric_Arrays(ar):#Takes a list of laten variable copy index exchange operations and the values of the observable variables as input as input 
                                     #Returns all configurations of observable variable values that have identical probabilities in the form of a list
    A=unpackbits(ar.view(uint8), axis=1, bitorder='little')
    return vstack((transpose(ar),\
     packbits(A[:,SwapX1X2], bitorder='little').view(uint32),\
     packbits(A[:,SwapY1Y2], bitorder='little').view(uint32),\
     packbits(A[:,SwapZ1Z2], bitorder='little').view(uint32),\
     packbits(A[:,SwapX1X2[SwapY1Y2]], bitorder='little').view(uint32),\
     packbits(A[:,SwapX1X2[SwapZ1Z2]], bitorder='little').view(uint32),\
     packbits(A[:,SwapY1Y2[SwapZ1Z2]], bitorder='little').view(uint32),\
     packbits(A[:,SwapX1X2[SwapY1Y2[SwapZ1Z2]]], bitorder='little').view(uint32),\
    ))
start = time.time()
A=Symmetric_Arrays(ar)
print('It took', time.time()-start, 'seconds.')
A.shape

It took 13.635999917984009 seconds.


(8, 16777216)

In [123]:
start = time.time()
A=A[:,unique(amin(A,axis=0))]
print('It took', time.time()-start, 'seconds.')
A.shape

It took 0.8059978485107422 seconds.


(8, 2123776)

In [ ]:
"""
AasBits=transpose(atleast_2d(ravel(A))).view(uint8)
AasBits=unpackbits(AasBits,axis=1, bitorder='little').view(bool_)
AasBits.shape
"""

In [254]:
arasbits=unpackbits(ar.view(uint8),axis=1, bitorder='little').view(bool_)

In [187]:
temparray=zeros_like(arasbits)
temparray[:,:12]=arasbits[:,23-array([0,1,6,7,8,9,14,15,16,17,22,23])]
A1=packbits(temparray, bitorder='little').view(uint32)
temparray=zeros_like(arasbits)
temparray[:,:12]=arasbits[:,23-array([23,22,17,16,15,14,9,8,7,6,1,0])] #DEFINETLY A MISTAKE HERE!!
A2=packbits(temparray, bitorder='little').view(uint32)
fromquadratictorow=amin(vstack((A1,A2)),axis=0)
len(unique(fromquadratictorow))

2080

In [210]:
"""
temparray=zeros_like(AasBits)
temparray[:,:12]=AasBits[:,23-array([0,1,6,7,8,9,14,15,16,17,22,23])]
rowencoding=reshape(packbits(temparray, bitorder='little').view(uint32),A.shape)
"""

In [249]:
temp=zeros(4**6,uint16);
u=unique(fromquadratictorow)
temp[u]=arange(0,len(u),1,uint16);
encoding=temp[fromquadratictorow]
encoding.shape

(16777216,)

In [252]:
sparsematrixdata=encoding[A]
sparsematrixdata.shape

(8, 2123776)

---UP TO THIS POINT EDITED---
---I'm not happy with the row construction. We need to know which of the 4^6 probabilities corresponds to which of the 2080 rows. This is best done by defining an encoding from 4096 to 2048, and then compressing.

In [4]:
def CorRowsNoSymmetry(ar):
    
    A=IntToArray32(ar)
    
    #A=A[:,[0,1,6,7,8,9,14,15,16,17,22,23]]
    A=A[:,[23,22,17,16,15,14,9,8,7,6,1,0]]

    A=ArrayToInt16(A)
    
    return A

def CorRows(ar):#Creates an array with all the rows that have a "1" in the corresponding column of the input array. the row symmetries are applied
                #by choosing the row with the smaller index among the symmetric rows
    
    A=IntToArray32(ar)
    
    A1=A[:,[0,1,6,7,8,9,14,15,16,17,22,23]]
    #A1=A[:,[14,9,8,7,6,1,0,23,22,17,16,15]]
    
    A1=ArrayToInt16(A1)
    
    A=A[:,[23,22,17,16,15,14,9,8,7,6,1,0]]
    
    A=ArrayToInt16(A)
    A=vstack((A,A1))
    A=amin(A,axis=0)
    
    return A
start = time.time()
A=CorRows(ar)
print('It took', time.time()-start, 'seconds.')

It took 3.7859010696411133 seconds.


In [5]:
def NewCols(ar):#Applies the symmetry condition to the columns by choosing the column with the smaller index among the symmetric columns
    
    Op_list=['X1_X2','Y1_Y2','Z1_Z2']
    
    A=Symmetric_Arrays(Op_list, ar)
    
    A=amin(A,axis=1)
    
    #A=A[:,0]
    
    return A
start = time.time()
A=NewCols(ar)
print('It took', time.time()-start, 'seconds.')

It took 143.67358303070068 seconds.


In [6]:
def remove_zero_rows(M):# Removes zero rows from sparse array
  M = csr_matrix(M)
  num_nonzeros = diff(M.indptr)
  return M[num_nonzeros != 0]

def drop_zero_columns(m):#removes zero columns from sparse array
    
    nz_cols, new_col = unique(m.col, return_inverse=True)
    m.col[:] = new_col
    m._shape = (m.shape[0], len(nz_cols))

    return m

def MDM():#Creates the MDM by combining the outputs of the previous functions in a coo sparse array
        
    ar=array([arange(4**12)]).T
        
    Rows=CorRows(ar)
    Columns=NewCols(ar)
        
    data=[1 for i in range(len(Rows))]
    
    m = coo_matrix((data, (Rows, Columns)),dtype=uint8)
    M=drop_zero_columns(m)
    M=remove_zero_rows(M)
    M = M.toarray()
    
    return M
start = time.time()
M=MDM()
print('It took', time.time()-start, 'seconds.')
print('The number of columns in the MDM:')
print(len(M[0]))
print('The number of rows in the MDM:')
print(len(M))

It took 154.16555166244507 seconds.
The number of columns in the MDM:
2123776
The number of rows in the MDM:
2080


In [8]:
def RowSymmetry(inte):#Takes a row as input
                      #Returns the symmetric row
    
    c=IntToList(inte,4,6)
    c=[c[1],c[0],c[3],c[2],c[5],c[4]]
    c=ListToInt(c,4)
    
    return c


def b(Data):#Creates the b vector by removing the symmetrical rows and using the input data with the correct multiplications of the probabilities.
    
    Checklist=[0 for i in range(4**6)]
    i=0
    b=[]
    while i in range(len(Checklist)):
        
        if Checklist[i] == 0:
            
            R=RowSymmetry(i)
            
            c=IntToList(i,4,6)
            c1=[c[0],c[1],c[2]]
            c1=ListToInt(c1,4)
            c2=[c[3],c[4],c[5]]
            c2=ListToInt(c2,4)
            
            c3=Data[c1]*Data[c2]
            
            b.append(c3)
            Checklist[R]=1
            
        i=i+1
    
    b=array([b]).T
    
    return b
Data=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
start = time.time()
b=b(Data)
print('It took', time.time()-start, 'seconds.')
print('The length of the vector b is:')
print(len(b))

It took 0.02775096893310547 seconds.
The lenght of the vector b is:
2080


In [9]:
def scipy_sparse_to_spmatrix(A):#Turns a scipy sparse array into a cvxopt sparse array
    coo = A.tocoo()
    SP = spmatrix(coo.data.tolist(), coo.row.tolist(), coo.col.tolist(), size=A.shape, tc='d')
    return SP

A=M
c=b
z=zeros((len(A[0]),1))#Creates the right side of the inequality y.M>=0 in the form of a vector
    
A=A.astype(double)#Changes the format of the input arrays in a way that cvxopt functions will accept
c=c.astype(double)
z=z.astype(double)
A=transpose(A)#Changes the rows with columns as this is how cvxopt accepts the input matrix M
A=coo_matrix(A)#Changes the input matrix M to sparse form so that the solver won't crash the python kernel

"""
A=matrix(A)
c=matrix(c)
z=matrix(z)
"""

A=scipy_sparse_to_spmatrix(A)#Turns the scipy sparse array M into cvxopt sparse array
c=matrix(c)#Turns numpy array objective function to cvxopt dense array
z=matrix(z)

sol=solvers.lp(c,A,z, solver='mosek')#Solves the linear program

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123776         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.87    
Optimizer terminated. Time: 1.13    


Interior-point solution summary
  Problem status  : DUAL_INFEASIBLE
  Solution status : DUAL_INFEASIBLE_CER
  Primal.  obj: -4.0900086883e-04   nrm: 2e+00    Viol.  con: 0e+00    var: 0e+00  

Basic solution summary
  Problem status  : DUAL_INFEASIBLE
  Soluti